In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
flipkart= pd.read_csv('/content/smartphones - smartphones.csv')

In [ ]:
flipkart_clean = pd.DataFrame()

# Summary for data

This smart mobile phones dataset contains 1,021 rows and 11 columns, providing detailed specifications of smartphones from multiple brands. The dataset covers essential aspects such as model, price, ratings, SIM and network support, processor details, RAM and storage, battery capacity, display specifications, camera configurations, memory card support, and operating system version. It offers a structured snapshot of smartphone features and pricing,

# Column Descriptions

1. `model`:- Name of the smartphone, including brand and series/model.
2. `price`:- Price of the phone (in INR).
3. `raing`:- Overall customer rating of the phone (out of 100).
4. `sim`:- Information about SIM support, network types (3G/4G/5G), VoLTE, Wi-Fi, NFC, and connectivity options.
5. `processor`:- Details about the chipset, number of cores, and clock speed of the processor.
6. `ram`:- Information on RAM size and inbuilt storage (ROM).
7. `battery`:- Battery capacity (mAh), charging speed (watts), and fast-charging support.
8. `display`:- Display size, resolution, refresh rate, and type of notch/punch-hole.
9. `camera`:- Camera setup including rear and front cameras with megapixel details.
10. `card`:- Memory card support availability and maximum expandable storage.
11. `os`:-Operating system version of the device (e.g., Android v13, iOS v16).


# 📊 Data Assessing

---

## ✅ Quality Issues  
(problems with accuracy, completeness, validity, consistency)

1. **`model`** – brand names written in inconsistent formats, *(consistency)*  
2. **`price`** – contains unnecessary **₹** symbol *(validity)*  
3. **`price`** – contains commas in numeric values *(validity)*  
4. **`price`** – outlier value (e.g., Namotel listed at ₹99) *(accuracy)*  
5. **`rating`** – missing values *(completeness)*  
6. **`processor`** – incorrect values for some Samsung phones (rows: 642, 647, 649, …) *(validity)*  
7. Non-phone device (`iPod`, row 756) present *(validity)*  
8. **`ram`** – incorrect values in multiple rows *(validity)*  
9. **`battery`** – incorrect or incomplete values in multiple rows *(validity)*  
10. **`display`** – sometimes refresh rate missing *(completeness)*  
11. **`display`** – incorrect values in multiple rows *(validity)*  
12. Foldable phones have scattered or inconsistent info *(validity)*  
13. **`camera`** – uses inconsistent words (“Dual”, “Triple”, “Quad”), front & rear separated by **&** *(consistency)*  
14. **`camera`** – incorrect values in many rows *(validity)*  
15. **`card`** – sometimes contains info about OS or Camera *(validity)*  
16. **`os`** – sometimes contains info about Bluetooth/FM radio *(validity)*  
17. **`os`** – incorrect values in rows (324, 378) *(validity)*  
18. **`os`** – version names written inconsistently (e.g., “Lollipop”) *(consistency)*  
19. Missing values in **camera, card, os** *(completeness)*  
20. **`price`** and **`rating`** stored as strings instead of numeric *(validity)*  

---

## 🧹 Tidiness Issues  
(structure/formatting problems – need splitting or restructuring)

1. **`sim`** → split into: `has_5G`, `has_NFC`, `has_IR_Blaster`  
2. **`ram`** → split into: `RAM`, `ROM`  
3. **`processor`** → split into: `processor_name`, `cores`, `CPU_speed`  
4. **`battery`** → split into: `capacity`, `fast_charging_available`, `wattage`  
5. **`display`** → split into: `screen_size`, `resolution_width`, `resolution_height`, `refresh_rate`  
6. **`camera`** → split into: `rear_camera`, `front_camera`  
7. **`card`** → split into: `card_supported`, `max_expandable_storage`  

---

 - some of the phones are not Smart phone they are `FEATURE PHONE`, we are removing those rows

🔍 *These issues were identified both manually and programmatically.*


In [ ]:
pd.set_option("display.max_rows", None)   # show all rows

In [ ]:
clean_df = pd.DataFrame()

#### As there are 1000 rows, it difficult to deal with them together, so we will take a bath of 300 and then next batch

These additional info found inside the `model_name` columns
* Parentheses:- ( )

RAM + ROM combinations:-
 - 8GB RAM + 256GB
 - 6GB RAM + 128GB
 - 3GB RAM + 6 B
 - B RAM + 128GB
 - 12GB RAM + 1TB
 - 128GB, 256GB. 512GB. 1TB
 - 6 B + 128GB (or any variant of messy “B RAM + X B/GB/TB”)

* Extra spaces:- Multiple consecutive spaces after replacements

In [ ]:
df = flipkart.copy()

# First we deal with CONSISTENCY issue

### `brand` column

In [ ]:
#---- there is extra info present inside (), but after careful examination, it is not useful for model_name and brand_name
mask = df['model'].str.contains(
    r"\((?!.*(?:GB|TB|20[1-2][0-9])).*?\)",
    regex=True
)

df_filtered = df.loc[mask, 'model']
print(df_filtered)


754    Apple iPod Touch (7th Gen)
829         Infinix Note 12 (G96)
Name: model, dtype: object


In [ ]:
def extract_brand_and_model(series):
    """
    Extract brand_name (first word) and model_name (rest of string) from a pandas Series.

    Cleaning done:
    - Remove parentheses containing GB, TB, or years (2019–2029)
    - Remove 4G/5G from model names
    - Strip extra spaces
    Returns two Series: brand_name, model_name
    """

    # --- Extract brand (first word) ---
    brand_name = series.str.split().str[0].str.strip().str.title()

    # --- Extract model name (remove brand and clean patterns) ---
    temp = series.str.split(' ', n=1).str[1].str.strip()  # remove brand

    # Remove parentheses containing GB/TB/year
    temp = temp.str.replace(r"\((?:[^)]*(?:GB|TB|20[1-2][0-9])[^)]*)\)", "", regex=True).str.strip()

    # Remove 4G or 5G
    temp = temp.str.replace(r"\s*(?:4G|5G)\b", "", regex=True).str.strip()

    # Validation: check if any extra patterns still exist
    gb_tb_year_pattern = r"\((?:[^)]*(?:GB|TB|20[1-2][0-9])[^)]*)\)"
    if temp.str.contains(gb_tb_year_pattern, regex=True).any() or temp.str.contains(r"\b(?:4G|5G)\b", regex=True).any():
        raise ValueError("There are still extra patterns left in the series!")

    return brand_name, temp


   # --- Usage Example ---
brand_name, model_name = extract_brand_and_model(df['model'])

# Value counts for brands
# print(brand_name.value_counts())

# Check if any parentheses left in model_name
print(model_name[model_name.str.contains(r"\([^)]*\)", regex=True)])

754    iPod Touch (7th Gen)
829           Note 12 (G96)
Name: model, dtype: object


In [ ]:
model_name.count()

np.int64(1020)

In [ ]:
#---- Inserting column back into the df ----
df.insert(0,'brand_name',brand_name)
df.insert(1,'model_name', model_name)

## `os` column

after running this fn there are still some issue left, like unnecesssary values, but those values are not present, so we cannot deal with them, we can replace them using some statistical rules.

In [ ]:
def clean_extract_os(df, os_col='os', card_col='card'):
    """
    Clean and extract OS name and version from a dataframe.

    Steps:
    1. Identify rows where the os column has incorrect values (like No, Bluetooth, Memory, Browser, 0.3, 1.3)
       and the card column contains valid OS info (Android, OS, iOS, HarmonyOS, Nucleus)
    2. Copy valid OS values from card to os at those locations
    3. Standardize 'HarmonyOS' to 'Harmony'
    4. Split os into os_name and os_version
    5. Insert os_name and os_version columns into the dataframe
    """

    # --- Identify rows with invalid os values but valid card info ---
    invalid_os_pattern = r'No|Bluetooth|Memory|Browser|0\.3|1\.3'
    valid_card_pattern = r'Android|OS|iOS|HarmonyOS|Nucleus'

    temp_df = df[
        df[card_col].str.contains(valid_card_pattern, na=False, case=False) &
        df[os_col].str.contains(invalid_os_pattern, regex=True, na=False, case=False)
    ]

    # --- Update os values from card column where needed ---
    df.loc[temp_df.index, os_col] = temp_df[card_col].str.strip().values

    # --- Standardize HarmonyOS ---
    df[os_col] = df[os_col].str.replace('HarmonyOS', 'Harmony', case=False, regex=True)

    # --- Split into os_name and os_version ---
    os_name = df[os_col].str.split(' ').str[0]
    os_version = df[os_col].str.split(' ').str[1].str.replace('v','', case=False)

    # --- Insert new columns ---
    df.insert(df.columns.get_loc(os_col)+1, 'os_name', os_name)
    df.insert(df.columns.get_loc('os_name')+1, 'os_version', os_version)

    return df

# --- Usage ---
df = clean_extract_os(df)



## `price` column :- this column only contains ','. And also converting column into decimal.

In [ ]:
def clean_price(df, price_col='price'):
    """
    Clean a price column by:
    1. Removing any non-numeric characters
    2. Converting to float
    3. Checking that all values are numeric
    """
    # Remove non-numeric characters
    cleaned = df[price_col].astype(str).str.replace(r'[^\d.]', '', regex=True)

    # Check if all values are numbers
    if not cleaned.str.replace('.', '', 1).str.isdigit().all():
        raise ValueError(f"Some values in '{price_col}' are not numeric after cleaning!")

    # Convert to float
    df[price_col] = cleaned.astype(float)

    return df

# --- Usage ---
df = clean_price(df)



## `Processor` column: there is a lot info that needed to be extracted, like brand name, number of cores, frequency of the CPU. Also some info of this column is in adjacent column, that needs to be adjusted too.

In [ ]:
import pandas as pd
import re

def clean_processor_series(series):
    """
    Clean a pandas Series of processor names:
    - Normalize spaces and title case
    - Fix typos (Sanpdragon → Snapdragon)
    - Apply custom mapping
    - Handle special processor naming cases
    """

    # --- Initial normalization ---
    series = series.str.split(',').str[0].str.strip()
    series = series.str.strip().str.title()
    series = series.str.replace(r'\s+', ' ', regex=True)

    # --- Mapping replacements ---
    mapping = {
        "Snapdragon 8 Gen1": "Snapdragon 8 Gen1",
        "Sanpdragon" : "Snapdragon",
        "Snapdragon 8+ Gen1": "Snapdragon 8+ Gen1",
        "Snapdragon 8 Gen2": "Snapdragon 8 Gen2",
        "Snapdragon 870": "Snapdragon 870",
        "Snapdragon 888": "Snapdragon 888",
        "Snapdragon 680": "Snapdragon 680",
        "Snapdragon 778G+": "Snapdragon 778G Plus",
        "Snapdragon Qm215": "Snapdragon QM215",
        "Snapdragon 8+ Gen 1": "Snapdragon 8+ Gen1",
    }
    series = series.replace(mapping)

    # --- Define row-wise cleaning function ---
    def clean_processor(name):
        if pd.isna(name):
            return name

        # Normalize spaces again
        name = re.sub(r'\s+', ' ', name).strip()

        # Remove "5G" suffix
        name = re.sub(r'\b5G\b', '', name, flags=re.IGNORECASE).strip()

        # Remove "Samsung" prefix
        name = re.sub(r'^Samsung\s+', '', name, flags=re.IGNORECASE).strip()

        # Special processor fixes
        special_cases = {
            r'(Apple\s+)?A13(\s+Bionic)?': "Bionic A13",
            r'Sc6531E': "Unisoc Sc6531E",
            r'Dimensity\s+8100-Max': "Dimensity 8100 Max",
            r'Snapdragon\s+Qm215': "Snapdragon QM215",
            r'Samsung\s+Exynos\s+7885': "Exynos 7885",
            r'Sc9863A': "Unisoc Sc9863A",
        }

        for pattern, replacement in special_cases.items():
            if re.fullmatch(pattern, name, flags=re.IGNORECASE):
                name = replacement
                break

        # Replace '-' with space
        name = name.replace('-', ' ')

        return name

    # Apply cleaning function
    cleaned_series = series.apply(clean_processor)

    # Optional check: confirm no empty strings
    if cleaned_series.isna().any():
        print("Warning: Some processor names are still NaN after cleaning!")

    return cleaned_series

# --- Usage ---
processor_clean = clean_processor_series(df['processor'])
print(processor_clean.value_counts())


processor
Snapdragon 695             53
Snapdragon 680             43
Dimensity 700              39
Snapdragon 8+ Gen1         39
Snapdragon 8 Gen1          36
Helio P35                  31
Dimensity 810              30
Snapdragon 8 Gen2          26
Snapdragon 870             23
Snapdragon 888             23
Snapdragon 778G            21
Helio G96                  21
Helio G35                  20
Bionic A15                 18
Helio G85                  18
Helio G99                  17
Helio G80                  14
Dimensity 920              13
Helio G25                  13
Snapdragon 865             13
Bionic A16                 12
Dimensity 1080             12
Snapdragon 778G Plus       12
Snapdragon 720G            12
Helio G37                  12
Unisoc Sc9863A             11
Helio G95                  11
Dimensity 1300             11
Dimensity 900              10
Helio A22                  10
Exynos 850                  9
No Wifi                     9
Octa Core Processor         9


### processor info is leaked into the RAM column, index 532, 611


In [ ]:
df['ram'][df['ram'].str.contains('MHz')]

,ram
532,"Single Core, 208 MHz Processor"
611,"Dual Core, 500 MHz Processor"
